Objectifs :
- Implémenter le bag of word ;
- Implémenter le TF-IDF.

## Importations des packages

In [15]:
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Importations des données

Les données sont issue de [cette base de données](https://cs.nyu.edu/~kcho/DMQA/).

Maintenant que nos outils sont chargés, nous allons charger nos données.

Cliquez sur le lien ci-dessous :

https://drive.google.com/drive/folders/12OmusfAUOcoLOCwEc--nfkKQ5eEozU45?usp=sharing

Cliquer droit sur le dossier data et appuyer sur ajouter à mon drive.

In [16]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


Les données sont maintenant dans votre environnement collab.

In [17]:
import os
print(os.listdir('gdrive/MyDrive/Exercice_1/Partie_1')[:10])

['0a3ff2f0a147c158845afa44d2a012064896566b.story', '0a3fff5779a8f7cfdde5d284a429ab89fd5e85df.story', '0a0f56ebc5a0a67ed18de79d99b40a42d8058d04.story', '0a3ad75d92c5bc2eccf2763df86afe5ddeffed75.story', '0a3f2400ba4e5cdf4b3638ae6fb60fdfa12a2680.story', '0a3f567efff9f0748b2758c9e8c17dc66beade04.story', '0a05b14962b2e73bbff82086762e0e23d32b359f.story', '0a1ad82d161d90d758240407cb8c8fcebff4a212.story', '0a4ec4d37683347ca62b53982d2c5f4efb86f444.story', '0a4b2d4ea5fb0625e3e747525062f0a85345e4df.story']


# Création de la base de données

In [18]:
cmpt = 0
dict_data = dict()

for file_name in os.listdir('gdrive/MyDrive/Exercice_1/Partie_1'):
    f = open('gdrive/MyDrive/Exercice_1/Partie_1/'+file_name, 'r')
    lst = ""
    for line in f:

       line.strip()
       line = line.replace("\n" ,'')
       line = line.replace("//" , '')
       line = line.replace("/" , '')
       if len(line) > 0 :
        lst += line

    dict_data[cmpt] = lst
    cmpt += 1


# Prétraitements

## En minuscule

In [19]:
dict_data_min = dict()
for k, v in dict_data.items():
  dict_data_min[k] = v.lower()

## Tokenization

In [20]:
dict_data_token = dict()
for k, v in dict_data_min.items():
  tokenizer = nltk.RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(v)
  dict_data_token[k] = tokens

## Stopwords

In [21]:
dict_data_stop = dict()
for k, v in dict_data_token.items():
  dict_data_stop[k] = [w for w in v if not w in list(nltk.corpus.stopwords.words())]

## Stemming

In [22]:
dict_data_stem = dict()
st = LancasterStemmer()

for index, doc in dict_data_stop.items():
  dict_data_stem[index] = [st.stem(w) for w in doc ]

# Bag of words

## Vecteur de vocabulaire

In [23]:
vocab = set()

for index, doc in dict_data_stem.items():
  for word in doc :
    vocab.add(word)

print(len(vocab))

4016


## Bag of words

In [37]:
# Création du Bag of Words pour chaque document
bow_documents = {}

for index, doc in dict_data_stem.items():
    # Construire le vocabulaire pour le document
    # vocab = set(doc) # The problem is here - vocab is a set
    vocab = list(set(doc))  # Convert vocab to a list

    # Initialiser un DataFrame avec le vocabulaire du document
    data = pd.DataFrame(0, index=[index], columns=vocab)

    # Remplir le Bag of Words pour ce document
    for word in doc:
        data.loc[index, word] += 1

    # Ajouter au dictionnaire des résultats
    bow_documents[index] = data

# Afficher les Bag of Words pour chaque document
for doc_id, bow in bow_documents.items():
    print(f"\nBag of Words pour le document {doc_id} :")
    print(bow)


Bag of Words pour le document 0 :
   threatened  hostess  clos  correct  lovin  rel  ireport  1985  ap  al  ...  \
0           1        1     1        1      1    1        1     1   2   1  ...   

   solo  firearm  disagree  1970s  truc  calib  ask  ago  spect  just  
0     1        3         1      1     1      1    1    1     23     2  

[1 rows x 288 columns]

Bag of Words pour le document 1 :
   145  read  cov  break  distribut  spin  act  bant  pag  calder  ...  morn  \
1    2     2    1      1          1     2    1     1    1       2  ...     1   

   aw  fox  forc  lat  big  caus  continu  april  control  
1   2    1     7    1    1     1        1      1        2  

[1 rows x 249 columns]

Bag of Words pour le document 2 :
   sec  weapon  nassau  pict  act  column  emmy  send  video  low  ...  \
2    2       1       1     1    1       1     1     1      1    1  ...   

   cameram  auth  wid  roy  caus  car  continu  april  30th  thing  
2        1     1    1    9     1    6    